In [1]:
! pip install multimodal-transformers

  Created wheel for multimodal-transformers: filename=multimodal_transformers-0.1.4a0-py3-none-any.whl size=20631 sha256=06d2f020afcb99658c99c35cc6b02f39eb0044508d60320f54397d96f6758321
  Stored in directory: c:\users\asus.laptop-evkq06qk\appdata\local\pip\cache\wheels\98\7d\6d\4398271d9112fbed70c0b1ddb6af3ccc25beddc9d7d5d625a1
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d90f908725b4ea8054c63475bfd564376496a326c320067a5642e9a7bd4eab24
  Stored in directory: c:\users\asus.laptop-evkq06qk\appdata\local\pip\cache\wheels\87\39\dd\a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built multimodal-transformers sacremoses


In [2]:
from dataclasses import dataclass, field
import json
import logging
import os
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Optional

import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoConfig,
    Trainer,
    EvalPrediction,
    set_seed
)

from transformers.training_args import TrainingArguments
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
from multimodal_transformers.data import load_data_from_folder
from multimodal_transformers.model import TabularConfig
from multimodal_transformers.model import AutoModelWithTabular
import warnings; warnings.simplefilter('ignore')

logging.basicConfig(level=logging.INFO)
os.environ['COMET_MODE'] = 'DISABLED'

In [6]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.
